This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [12]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'seMG3m985xWrhRwXNyZ8'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [13]:
# First, import the relevant modules
import requests


Note: API's can change a bit with each version, for this exercise it is reccomended to use the "V3" quandl api at `https://www.quandl.com/api/v3/`

In [14]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key='+API_KEY\
+'&start_date=2018-05-18&end_date=2018-05-18'
r = requests.get(url)

In [15]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_data = r.json()
print(json_data)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2018-05-18', 'end_date': '2018-05-18', 'frequency': 'daily', 'data': [['2018-05-18', None, 59.0, 57.1, 57.95, None, 81077.0, 4707479.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [16]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key='+API_KEY\
+'&start_date=2017-01-01&end_date=2017-12-31'
r = requests.get(url)

2. Convert the returned JSON object into a Python dictionary.

In [17]:
data = r.json()

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [18]:
#Find the index of opening price
idx = data['dataset_data']['column_names'].index('Open')
#Some opening prices were None
open_price = [day[idx] for day in data['dataset_data']['data'] if day[idx]]
print('Highest and lowest opening prices in 2017 were {} and {}'.format(max(open_price), min(open_price)))

Highest and lowest opening prices in 2017 were 53.11 and 34.0


4. What was the largest change in any one day (based on High and Low price)?

In [19]:
#Find the index of high and low prices
idx_high = data['dataset_data']['column_names'].index('High')
idx_low = data['dataset_data']['column_names'].index('Low')
#Calculate change
change = [day[idx_high]-day[idx_low] for day in data['dataset_data']['data']]
print('The largest change in any one day in 2017 was {:.2f}'.format(max(change)))

The largest change in any one day in 2017 was 2.81


5. What was the largest change between any two days (based on Closing Price)?

In [20]:
#Find the index of closing price
idx = data['dataset_data']['column_names'].index('Close')
#Extract closing price
closing = [day[idx] for day in data['dataset_data']['data']]
#Calculate change between everyday and the day before
change_two_days = [abs(closing[index]-closing[index-1]) for index, value in enumerate(closing) if index > 0]
print('The largest chagne between any two days in 2017 was {:.2f}'.format(max(change_two_days)))

The largest chagne between any two days in 2017 was 2.56


6. What was the average daily trading volume during this year?

In [21]:
#Find the index of volume
idx = data['dataset_data']['column_names'].index('Traded Volume')
#Extract trading volume
volume = [day[idx] for day in data['dataset_data']['data']]
#Since no third party packages are allowed, calculte the average the old fashioned way.
average_volume = sum(volume)/len(volume)
print('The average daily trading volumne in 2017 was {:.2f}'.format(average_volume))

The average daily trading volumne in 2017 was 89124.34


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [22]:
#Sort volume first
volume = sorted(volume)
#Find the median based on definition of median
if len(volume) % 2 == 1:
    median_volume = volume[len(volume)//2]
else:
    median_volume = (volume[len(volume)//2] + volume[len(volume)//2-1]) / 2
print('The median trading volume in 2017 was {:.2f}'.format(median_volume))

The median trading volume in 2017 was 76286.00
